In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import prince
import importlib

import matplotlib.cm as cm
import numpy as np
from matplotlib.patches import Patch
import seaborn as sns
import matplotlib.lines as mlines
import matplotlib.cm as cm
import matplotlib.patches as mpatches

from networkx.algorithms import bipartite


import community as community_louvain
from netgraph import Graph

import sys
sys.path.insert(0, '../Utility files')
import corg
from corg import DiscoverDimension
import seaborn as sn
from corg import BenchmarkDimension
from corg import DiscoverDimension


import utils2
from utils2 import *
import graphfunk
from graphfunk import *


/tmp/ipykernel_1070120/474417774.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


See this link for setting up the facebook marketing API code:
- https://github.com/carolcoimbra/facebook-ads/blob/master/example.ipynb 

# What Markers to Include?

In this notebook, I'll try to identify what markers that make sense to include in our analysis. 
- We identify the dimension that is best at classifying/separating markers of clear difference in SES.
- We do this via the CORG method for different CA fits (varying the network to include or leave out certain markers)

### Load data

- The data loaded here contains a row for each follower relationship a brand has. It includes the original type column, as well as the changed column (condensed from 12 to 4 types.)
- Additionally, it has been filtered to only include markers with more than 100 followers

In [17]:
# load data
path = '/home/livtollanes/NewData/french/french_edgelist_marker_threshold.csv'

req_cols = ['marker_id', 'follower_id', 'twitter_name','followers','french_followers', 'type', 'type2']
dtypes = {'marker_id': 'object',
          'follower_id': 'object',
          'twitter_name': 'object',
          'followers': 'int64',
          'french_followers': 'int64',
          'type': 'object',
          'type2': 'object'}


edgelist_CORG = pd.read_csv(path, usecols=req_cols, dtype=dtypes)

### Create a list of high and low SES

In [5]:
edgelist_CORG.head()

,marker_id,follower_id,twitter_name,type,followers,french_followers,type2
0,19976004,1193293102162862080,Mediapart,media,3079643,64182,information
1,19976004,2884619299,Mediapart,media,3079643,64182,information
2,19976004,782999494703054848,Mediapart,media,3079643,64182,information
3,19976004,124458686,Mediapart,media,3079643,64182,information
4,19976004,2960220189,Mediapart,media,3079643,64182,information


In [12]:
edgelist_CORG['french_followers'].max()

64182

- Within each of the 12 brand types, identify brands that are either high or low SES.

In [5]:
# load data
path = '/home/livtollanes/NewData/french/french_edgelist.csv'

req_cols = ['marker_id', 'follower_id', 'twitter_name','followers','type']
dtypes = {'marker_id': 'object',
          'follower_id': 'object',
          'twitter_name': 'object',
          'followers': 'int64',
          'type': 'object'}


edgelist_old = pd.read_csv(path, usecols=req_cols, dtype=dtypes)


edgelist_old['french_followers'] = edgelist_old.groupby('marker_id')['marker_id'].transform('size')

utils2.check_types(edgelist_old, 'type', 'french_followers')

Number of unique values in 'type': 12

type = Lycées professionels:
  Min 'french_followers': 9
  Max 'french_followers': 45

type = chain restaurants:
  Min 'french_followers': 121
  Max 'french_followers': 5716

type = clubs de football:
  Min 'french_followers': 139
  Max 'french_followers': 21482

type = commerce:
  Min 'french_followers': 171
  Max 'french_followers': 16142

type = ecoles de commerce:
  Min 'french_followers': 358
  Max 'french_followers': 4128

type = grande distribution:
  Min 'french_followers': 31
  Max 'french_followers': 8662

type = luxe vetements et malls:
  Min 'french_followers': 2
  Max 'french_followers': 9385

type = magazine:
  Min 'french_followers': 1
  Max 'french_followers': 40800

type = media:
  Min 'french_followers': 132
  Max 'french_followers': 64182

type = party:
  Min 'french_followers': 6
  Max 'french_followers': 17275

type = sport:
  Min 'french_followers': 2
  Max 'french_followers': 7885

type = universities:
  Min 'french_follower

Just double checked. We have only 11 types in the newly loaded df. This is because all Lycées Professionels in the french edgelist had less than 100 followers.

In [21]:
edgelist_CORG['twitter_name'].nunique()

223

In [22]:
#Separate my data frame into dictionary of types
def create_dict(df, key_col, value_col):
    unique_df = df.drop_duplicates(subset=[key_col, value_col])
    return unique_df.groupby(key_col)[value_col].apply(list).to_dict()


df_dict = create_dict(edgelist_CORG, 'type', 'twitter_name')
df_dict2 = create_dict(edgelist_CORG, 'type2', 'twitter_name')

In [23]:
for key, value in df_dict.items():
    value = list(set(value))
    print(f"Key: {key}")
    print(f"Value: {value}\n")

Key: Lycées professionels
Value: ['PSVLaTournelle', 'lpovanicet', 'LaCoudouliere', 'GilbertLycee', 'VaubanLycee', 'LPHurleventBLF', 'LyceeProTissie']

Key: chain restaurants
Value: ['brioche_doree', 'pizzahutfr', 'bigmammagroup', 'QuickFrance_', 'KFCFrance', 'BurgerKingFR', 'McDonaldsFrance']

Key: clubs de football
Value: ['OM_Officiel', 'ClermontFoot', 'OL', 'estac_officiel', 'LOSC_EN', 'PSG_inside', 'ogcnice', 'ToulouseFC', 'StadeDeReims', 'ACAjaccio', 'AS_Monaco', 'RCSA', 'FCLorient', 'FCNantes', 'RCLens', 'AJA', 'AngersSCO', 'MontpellierHSC']

Key: commerce
Value: ['vinted', 'Darty_Officiel', 'Fnac', 'Decathlon', 'AmazonFrance', 'Cdiscount', 'rueducommerce', 'boulanger']

Key: ecoles de commerce
Value: ['SKEMA_BS', 'inseec_ge', 'essec', 'IESEG', 'esdes_BS', 'Psbeduparis', 'ESCP_bs', 'EDHEC_BSchool', 'EMLYON']

Key: grande distribution
Value: ['CasinoEnseignes', 'Supermarche_G20', 'CarrefourFrance', 'picardsurgeles', 'lidlfrance', 'AUCHAN_France', 'marksandspencer', 'intermarche', 

In [24]:
import csv

# Create a csv file to use for labeling 
rows = []
for key, values in df_dict.items():
    for value in values:
        row = {'type': key, 'twitter_name': value, 'high': '', 'low': ''}
        rows.append(row)

# Now we write the rows to a CSV file
keys = ['type', 'twitter_name', 'high', 'low']

with open('/home/livtollanes/10.jan-thesis/labeldata/labeled_markers.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(rows)

The fact that we're using reasoning to determine our validation set here is slightly flawed.

In [ ]:
/home/livtollanes/10.jan-thesis/labeldata

## CORG for assessing models

In [7]:
import corg
from corg import DiscoverDimension
import seaborn as sn
from corg import BenchmarkDimension
from corg import DiscoverDimension
